In [72]:
import os
import cv2
import torch
import numpy as np
import torch.nn as nn
from torchvision import models
from torchvision import datasets, transforms

from models.resnet import resnet18

In [73]:
def get_device():
    if torch.cuda.is_available():
        device = torch.device("cuda")
    elif torch.backends.mps.is_available():
        device = torch.device("mps")
    else:
        device = torch.device("cpu")
    return device

device = get_device()
print(f"device is: {device}")

device is: cuda


In [74]:
# load the trained model
def get_resnet18(num_classes = 29, pretrained=False):
    model = resnet18(pretrained)
    model.fc = nn.Linear(model.fc.in_features, num_classes)
    return model
num_classes = 29
model_path = 'resnet18.pth' 
model = get_resnet18(num_classes, pretrained=False)
if os.path.exists(model_path):
    model.load_state_dict(torch.load(model_path, map_location='cpu'))
model = model.to(device)
model.eval()

C:\Users\Benjamin\AppData\Local\Temp\ipykernel_23232\3364409672.py:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_locatio

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [75]:
# get class names
def get_class_names(root_train = './data/train'):
    test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

    test_set = datasets.ImageFolder(root_train, transform = test_transform)
    class_names = test_set.classes
    return class_names
class_names = get_class_names(root_train = './data/train')
print(class_names)
assert len(class_names) == num_classes

['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'del', 'nothing', 'space']


In [76]:
# transform
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

In [77]:
# start webcam
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Could not open webcam.")
    exit()
print("Press 'q' to quit.")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    h, w, _ = frame.shape

    # define a region of interest
    roi_size = 300  
    cx, cy = w // 2, h // 2

    x1 = max(cx - roi_size // 2, 0)
    y1 = max(cy - roi_size // 2, 0)
    x2 = min(cx + roi_size // 2, w)
    y2 = min(cy + roi_size // 2, h)

    roi = frame[y1:y2, x1:x2]
    input_tensor = transform(roi).unsqueeze(0).to(device)

    with torch.no_grad():
        outputs = model(input_tensor)
        _, predicted = torch.max(outputs, 1)
        label = class_names[predicted.item()]

    # draw ROI and label
    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
    overlay = frame.copy()
    cv2.putText(overlay, label, (30, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255, 255, 255), 3)
    alpha = 0.75
    frame = cv2.addWeighted(overlay, alpha, frame, 1 - alpha, 0)
    cv2.imshow("aSL real-time recognition", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
   

cap.release()
cv2.destroyAllWindows()


Press 'q' to quit.
